# 🗄️ Advanced SQL – Day 18 (Subqueries & Views)

**Goal:**  
Learn how to:  
- Use **subqueries** (queries inside queries)  
- Create and query **views** for reusability  

#  1. Setup Database

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2. Create Tables

In [2]:
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    major TEXT
)
""")

cursor.execute("""
CREATE TABLE courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT,
    credits INTEGER
)
""")

cursor.execute("""
CREATE TABLE enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    grade TEXT,
    FOREIGN KEY(student_id) REFERENCES students(student_id),
    FOREIGN KEY(course_id) REFERENCES courses(course_id)
)
""")

# 3. Insert Sample Data

In [3]:
students_data = [
    (1, "Zeeshan", 21, "Computer Science"),
    (2, "Samiya", 22, "Information Tech"),
    (3, "Farhan", 20, "Mathematics"),
    (4, "Meera", 23, "Statistics"),
    (5, "Zara", 21, "Computer Science")
]

courses_data = [
    (101, "Database Systems", 3),
    (102, "Machine Learning", 4),
    (103, "Probability & Stats", 3)
]

enrollments_data = [
    (1, 1, 101, "A"),
    (2, 1, 102, "B"),
    (3, 2, 101, "A"),
    (4, 3, 103, "B"),
    (5, 4, 102, "A"),
    (6, 4, 103, "A"),
    (7, 5, 101, "B")
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?, ?)", students_data)
cursor.executemany("INSERT INTO courses VALUES (?, ?, ?)", courses_data)
cursor.executemany("INSERT INTO enrollments VALUES (?, ?, ?, ?)", enrollments_data)
conn.commit()


# 4. Subquery Example 1 – Students Above Average Age

In [4]:
query = """
SELECT name, age
FROM students
WHERE age > (SELECT AVG(age) FROM students)
"""
pd.read_sql(query, conn)

,name,age
0,Samiya,22
1,Meera,23


# 5. Subquery Example 2 – Students Enrolled in "Database Systems"

In [5]:
query = """
SELECT name
FROM students
WHERE student_id IN (
    SELECT student_id
    FROM enrollments
    WHERE course_id = (SELECT course_id FROM courses WHERE course_name = 'Database Systems')
)
"""
pd.read_sql(query, conn)

,name
0,Zeeshan
1,Samiya
2,Zara


# 6. Creating a View

In [6]:
cursor.execute("""
CREATE VIEW student_course_grades AS
SELECT s.name, c.course_name, e.grade
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
""")

In [7]:
pd.read_sql("SELECT * FROM student_course_grades", conn)

,name,course_name,grade
0,Zeeshan,Database Systems,A
1,Zeeshan,Machine Learning,B
2,Samiya,Database Systems,A
3,Farhan,Probability & Stats,B
4,Meera,Machine Learning,A
5,Meera,Probability & Stats,A
6,Zara,Database Systems,B


# 7. Query View with Filtering

In [8]:
pd.read_sql("SELECT * FROM student_course_grades WHERE grade = 'A'", conn)

,name,course_name,grade
0,Zeeshan,Database Systems,A
1,Samiya,Database Systems,A
2,Meera,Machine Learning,A
3,Meera,Probability & Stats,A


# ✅ Summary (Day 18 – Part 2):

- Learned subqueries (IN, nested queries for filtering)
- Created a view to simplify repeated JOIN queries
- Queried the view with filtering conditions
- Built SQL that looks much closer to real-world projects 🎯